In [383]:
import pandas as pd

data = pd.read_csv('train.txt', delimiter='\t')
del data['id']
data.loc[:,'label'] = data.loc[:,'label'].map({'others':0, 'happy':1, 'sad':2, 'angry':3})

features = data[['turn1','turn2','turn3']]
labels = data[['label']]

text = features.turn1+' '+features.turn2+' '+features.turn3
sentences = list(text)
sentences

["Don't worry  I'm girl hmm how do I know if you are What's ur name?",
 'When did I? saw many times i think -_- No. I never saw you',
 'By by Google Chrome Where you live',
 'U r ridiculous I might be ridiculous but I am telling the truth. U little disgusting whore',
 'Just for time pass wt do u do 4 a living then Maybe',
 "I'm a dog person youre so rude Whaaaat why",
 'So whatsup Nothing much. Sitting sipping and watching TV. How abt u? What are you watching on tv?',
 'Ok ok im back!! So, how are u',
 'Really? really really really really really Y saying so many times...i can hear you',
 'Bay in the bay 😘 love you',
 'I hate my boyfriend you got a boyfriend? Yes',
 'I will do night. Alright. Keep me in loop. Not giving WhatsApp no.',
 'Sure go ahead Many thanks once again! Love you too',
 "Bad Bad bad! That's the bad kind of bad. I have no gf",
 'Ok get it...... I made it an option Ok',
 "Money money and lots of money😍😍 I need to get it tailored but I'm in love with it 😍 😁😁",
 'My gf l

In [384]:
from bs4 import BeautifulSoup
import requests

from collections import ChainMap

def extract_emoticons_and_emojis(): 
    url = 'https://en.wikipedia.org/wiki/List_of_emoticons'
    res = requests.get(url)

    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    
    tables = soup.findAll('table', {'class':'wikitable'})
    
    descr_to_idx = get_descr_to_idx()
    emt_side, emj_side = scrape_emoticons_and_emojis_from_table(tables[0], descr_to_idx['side']) 
    emt_side2, emj_side2 = scrape_emoticons_and_emojis_from_table(tables[1], descr_to_idx['side2']) 
    emt_up, _ = scrape_emoticons_and_emojis_from_table(tables[2], descr_to_idx['up'], 0)
    emt_east, emj_east = scrape_emoticons_and_emojis_from_table(tables[4], descr_to_idx['east'])
    emt_east_oth, emj_east_oth = scrape_emoticons_and_emojis_from_table(tables[5], descr_to_idx['east_oth'])
    emt_2ch, _ = scrape_emoticons_and_emojis_from_table(tables[6], descr_to_idx['2-ch'],0)
    freq_emojis = scrape_freq_emojis(tables[10], tables[11], descr_to_idx['freq_emojis1'], descr_to_idx['freq_emojis2'])
    
    emoticonbook = [emt_side, emt_side2, emt_up, emt_east, emt_east_oth, emt_2ch]
    emojibook = [emj_side, emj_side2, emj_east, emj_east_oth]

    emoticons_mapper = dict()
    for book in emoticonbook:
        emoticons_mapper = dict(ChainMap({}, emoticons_mapper, book))
    emojis_mapper = dict()
    for book in emojibook:
        emojis_mapper = dict(ChainMap({}, emojis_mapper, book))
    emojis_mapper = dict(ChainMap({}, freq_emojis, emojis_mapper)) 
    
    return emoticons_mapper, emojis_mapper
    
def scrape_emoticons_and_emojis_from_table(table, descr_to_idx, emoji_flag=1):
    trs = table.find_all('tr')[1:]
    emoticonbook = dict()
    emojibook = dict()
    for i, tr in enumerate(trs):
        tds = tr.find_all('td')
        descr = tds[-1].text
        emoticons = []
        for td in tds[:-1-emoji_flag]:
            emoticons += [string.replace('\n','') for string in [c.string for c in td.contents] if string]
        emoticons = [emoticon for emoticon in emoticons if emoticon!='' and emoticon!=' ']

        for emoticon in emoticons:
            if descr_to_idx[i] != '' and descr_to_idx[i] != ' ':
                if emoticon not in emoticonbook.keys():
                    emoticonbook[emoticon] = descr_to_idx[i]
        if emoji_flag:
            emojis = tds[-2].text
            emojis = [emoji for emoji in emojis if emoji!='️' and emoji!='—' and emoji!='\n' and emoji!='️ ']
            for emoji in emojis:
                if descr_to_idx[i] != '' and descr_to_idx[i] != ' ':
                    if emoji not in emojibook.keys():
                        emojibook[emoji] = descr_to_idx[i]
    return emoticonbook, emojibook

def scrape_freq_emojis(table1, table2, descr_to_idx1, descr_to_idx2):
    freq_emojis = dict()
    trs = table1.find_all('td')[19:-1]
    trs = [tr.text.replace('\n','') for tr in trs]
    for i, emoji in enumerate(trs):
        if descr_to_idx1[i] != '':
            freq_emojis[emoji] = descr_to_idx1[i]
    
    trs = table2.find_all('td')[32:68]
    trs = [tr.text.replace('\n','') for tr in trs]
    for i, emoji in enumerate(trs):
        if descr_to_idx2[i] != '':
            freq_emojis[emoji] = descr_to_idx2[i]
            
    return freq_emojis

def get_descr_to_idx():
    descr_to_idx = dict()
    descr_to_idx['side'] = [
        'smile','laugh','smile','frown','crying','laugh','disgust, sadness','surprise','kiss','wink','playful',
        'annoyed','indecision','embarassed','sealed lips','innocent','evil','bored','disappointed',
        'partied all night','confused','sick','dumb','disapproval','nervous','guilty'
    ]
    descr_to_idx['side2'] = [
        'rose', '', 'santa claus', '', '', '', 'sad', 'love'
    ]
    descr_to_idx['up'] = [
        'fish', 'cheer', 'cheerleader', 'lennon', 'disgust, sadness', 'surprise', 'annoyed', 'high five', 
        'crab'
    ]
    descr_to_idx['east'] = [
        'troubled', 'baby', 'nervous', 'smoking', 'sleeping', 'wink', 'confused', 'ultraman', '', 'joyful', 
        'respect', 'questioning', 'sad', 'shame', 'tired', 'cat', 'looking down', 'giggling', 'confusion',
        'facepalm', 'laugh', 'waving', 'alien', 'excited', 'amazed', '', 'laughing', '', '', '', 'music',
        'disappointed', 'eyeglasses', 'jotting note', 'happy', 'grinning', 'surprised', 'infatuation', 'surprised',
        'dissatisfied', 'mellow', 'snubbed', 'kiss', 'studying', 'joy', 'cute'
    ]
    descr_to_idx['east_oth'] = [
        'bubbles', 'tea', 'star', 'fish', 'octopus', 'snake', 'bat', 'tadpole', 'bomb', 'despair', 'table flip', 
        ''
    ]
    descr_to_idx['2-ch'] = [
        'respect', 'snubbed', 'perky', 'salute', 'terribly sad', 'peace', 'irritable', 'angry', 'yelling', 
        'surprised','don\'t know answer', 'carefree', 'indifferent', 'shocked', 'happy', 'carefree', 'spook', 
        'surprised', 'jog cheek', 'amazed', 'smoking', 'cheers', 'intuition', 'friendly', 'lonely', 'depressed', 
        'thinking', 'impatience', 'whispers', 'money', 'sliding on belly', 'unforeseen', 'don\'t need it', 
        'come on', 'mocking', 'bad', 'goofing', 'sad', 'not convincing', 'simper', 'deflagged', 'happy', 'happy', 
        'despair', '', '', '', 'distaste', 'shouting', 'asleep', 'kick', 'discombobulated', 'running', 'happy', 
        'happy', 'shocked', 'angry', 'do it', '', 'angel'
    ]
    descr_to_idx['freq_emojis1'] = [
        'smile', 'smile', 'laugh', 'smile', 'smile', 'sweat', 'laugh', 'angel', 'evil smile', 'wink', 'smile', 
        'smile', 'smile', 'smile', 'smile', 'smirk', '', 'neutral', 'disappointed', 'disappointed', 
        'disappointed', 'worried', 'worried', 'sad', 'kiss', 'kiss', 'kiss', 'kiss', 'smile', 'smile, wink', 
        'smile', 'sad', 'sad', '', 'angry', 'angry', 'worried', 'worried', 'angry', 'worried', 'worried', 
        'worried', 'worried', 'worried', 'disappointed', 'worried', 'nervous', 'crying', 'surprised', 
        'surprised', '', 'worried', 'worried', 'surprised', 'surprised, sad', 'sleeping', 'sleeping', 
        'speechless', 'sick', 'laugh', 'laugh','smile', 'smile', 'smirk', 'kiss', 'angry', 'worried', '', 
        'worried', 'sad', 'smile', 'neutral', 'thinking','bad', 'ok', 'bowing', 'bad', 'surprised', 'bad', 
        'neutral', 'good', 'frown', 'frown', 'please'
    ]
    descr_to_idx['freq_emojis2'] = [
        'love', 'love', 'pinch', 'quiet', 'smile, money', 'sick', 'smile', 'thinking', 'sick', 'robot', 
        'smile', 'agree', 'agree', 'bye', 'agree', 'agree', 'agree', 'hope', 'agree', '', 'smile', 'smile',
        'sick', 'laugh', 'drooling', 'lying', 'disappointed', 'sneezing', 'thinking', 'smile', 'laugh, wink',
        'quiet', 'angry', 'laugh', 'sick', 'surprised'
    ]
    return descr_to_idx

def apply_emo_mapper(sent, mapper):
    new_sent = sent
    for key in mapper.keys():
        new_sent = new_sent.replace(key, ' '+mapper[key]+' ')
    return new_sent

In [360]:
from tokenizer import tokenizer
T = tokenizer.TweetTokenizer(regularize=True)
for sentence in sentences:
    new_sent = ' '.join(T.tokenize(sentence)) 

lower_sentences = list(map(lambda s: s.lower(), sentences))

emoticons_mapper, emojis_mapper = extract_emoticons_and_emojis()

sentences_without_emo = []
for sentence in lower_sentences:
    # removing emoticons
    new_sentence = apply_emo_mapper(sentence, emoticons_mapper)
    # removing emojis
    new_sentence = apply_emo_mapper(new_sentence, emojis_mapper)
    sentences_without_emo.append(new_sentence)


In [361]:
abbreviations = {
    "\$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

import re
sentences_without_abbreviations = []
for sentence in sentences_without_emo:
    new_sentence = sentence
    for abbr in abbreviations.keys():
        new_sentence = re.sub(r'^'+abbr+r'[^a-zA-Z\d]', abbreviations[abbr]+' ', new_sentence)
        new_sentence = re.sub(r'[^a-zA-Z\d]'+abbr+r'$', ' '+abbreviations[abbr], new_sentence)
        new_sentence = re.sub(r'[^a-zA-Z\d]'+abbr+r'[^a-zA-Z\d]', ' '+abbreviations[abbr]+' ', new_sentence)
        #new_sentence = new_sentence.replace(' '+abbr+' ',' '+abbreviations[abbr]+' ')
    sentences_without_abbreviations.append(new_sentence)
sentences_without_abbreviations

["don't worry  i'm girl hmm how do i know if you are what's ur name?",
 'when did i? saw many times i think -_- no. i never saw you',
 'by by google chrome where you live',
 'you r ridiculous i might be ridiculous but i am telling the truth. you little disgusting whore',
 'just for time pass wt do you do 4 a living then maybe',
 "i'm a dog person youre so rude whaaaat why",
 'so whatsup nothing much. sitting sipping and watching tv. how abt you  what are you watching on tv?',
 'ok ok im back!! so, how are you',
 'really? really really really really really y saying so many times...i can hear you',
 'bay in the bay  kiss  love you',
 'i hate my boyfriend you got a boyfriend? yes',
 'i will do night. alright. keep me in loop. not giving whatsapp no.',
 'sure go ahead many thanks once again! love you too',
 "bad bad bad! that's the bad kind of bad. i have no gf",
 'ok get it...... i made it an option ok',
 "money money and lots of money smile  smile  i need to get it tailored but i'm in lo

In [385]:
import re

repeat_pattern = re.compile(r'(\w)\1+')
match_substitution = r'\1\1'

sentences_without_elongations = []
for sentence in sentences_without_abbreviations:
    new_sentence = sentence
    new_sentence = repeat_pattern.sub(match_substitution, new_sentence)
    sentences_without_elongations.append(new_sentence)
sentences_without_elongations

["don't worry  i'm girl hmm how do i know if you are what's ur name?",
 'when did i? saw many times i think -_- no. i never saw you',
 'by by google chrome where you live',
 'you r ridiculous i might be ridiculous but i am telling the truth. you little disgusting whore',
 'just for time pass wt do you do 4 a living then maybe',
 "i'm a dog person youre so rude whaat why",
 'so whatsup nothing much. sitting sipping and watching tv. how abt you  what are you watching on tv?',
 'ok ok im back!! so, how are you',
 'really? really really really really really y saying so many times...i can hear you',
 'bay in the bay  kiss  love you',
 'i hate my boyfriend you got a boyfriend? yes',
 'i will do night. alright. keep me in loop. not giving whatsapp no.',
 'sure go ahead many thanks once again! love you too',
 "bad bad bad! that's the bad kind of bad. i have no gf",
 'ok get it...... i made it an option ok',
 "money money and lots of money smile  smile  i need to get it tailored but i'm in love

In [379]:
from tokenizer import tokenizer
import hunspell

T = tokenizer.TweetTokenizer(regularize=True)
hobj = hunspell.HunSpell('/usr/share/hunspell/en_US.dic', '/usr/share/hunspell/en_US.aff')

sentences_without_misspellings = []
for sentence in sentences_without_elongations:
    new_sentence = sentence
    for tok in T.tokenize(new_sentence):
        suggestions = hobj.suggest(tok)
        if not hobj.spell(tok) and tok.isalnum() and len(tok)>2 and len(suggestions)>0 and (not('h' in tok)):
            new_sentence = new_sentence.replace(tok, suggestions[0]) 
    sentences_without_misspellings.append(sentence)
            #print(tok, ' >> ', hobj.suggest(tok)[0])
    #new_sentence = ' '.join(tokens) 
    #sentences_without_misspellings.append(new_sentence)


#for sentence in sentences_without_elongations:



abt  >>  bat
bys  >>  bus
bcoz  >>  cozy
dont  >>  font
wat  >>  eat
ress  >>  reds
favourite  >>  favorite
andwrite  >>  and write
india  >>  India
didnt  >>  dint
wlcm  >>  LCM
ntng  >>  Ting
panty  >>  pant
abt  >>  bat
goa  >>  Goa
india  >>  India
ofcorse  >>  Scorsese
nekde  >>  needed
pls  >>  pl
asmita  >>  Tasmania
realky  >>  really
wat  >>  eat
liee  >>  lii
ected  >>  acted
ferari  >>  Ferrari
donot  >>  donor
appatasiri  >>  apparition
autocorrect  >>  auto correct
gobar  >>  lobar
dont  >>  font
necessar  >>  necessary
siri  >>  sir
dont  >>  font
irritatingne  >>  irritating
arent  >>  rent
aftar  >>  afar
dont  >>  font
iam  >>  aim
badroom  >>  bedroom
noo  >>  new
plz  >>  pl
orignal  >>  original
albace  >>  alb ace
esquise  >>  esquire
friendzone  >>  friend zone
friendzone  >>  friend zone
colgate  >>  Colgate
doesnt  >>  doesn't
reson  >>  resewn
lofe  >>  life
exjactly  >>  exactly
telugu  >>  Telugu
yaa  >>  ya
talisa  >>  tallish
robb  >>  rob
pict  >>  Pict
do

nop  >>  bop
mso  >>  mos
gud  >>  good
dont  >>  font
coz  >>  cox
annum  >>  annul
attrocity  >>  atrocity
panrel  >>  panel
srs  >>  rs
ronaldo  >>  Ronald
plz  >>  pl
paul  >>  Paul
vin  >>  bin
disel  >>  diesel
txtng  >>  texting
tym  >>  tum
bitcg  >>  bitch
knw  >>  kn
coz  >>  cox
knw  >>  kn
abt  >>  bat
dont  >>  font
srry  >>  sorry
kirollos  >>  rollicks
kero  >>  keno
diu  >>  die
pls  >>  pl
jus  >>  us
coz  >>  cox
spoked  >>  spiked
saturday  >>  Saturday
ofcouse  >>  coffeehouse
wiollbe  >>  Wilbert
monday  >>  Monday
dont  >>  font
bcoz  >>  cozy
aww  >>  awe
knw  >>  kn
opyrtc  >>  copyright
tuesday  >>  Tuesday
cbbyb  >>  cabby
dnt  >>  dent
biryani  >>  binary
biriyani  >>  Britannia
lanation  >>  la nation
wats  >>  WATS
yar  >>  ray
fucken  >>  turducken
tvm  >>  tam
mayb  >>  may
eid  >>  ad
lanja  >>  ganja
ame  >>  am
byr  >>  bur
byy  >>  buy
rady  >>  rad
plz  >>  pl
fuckoff  >>  kickoff
santa  >>  Santa
claus  >>  Claus
mee  >>  mi
kanna  >>  kana
linkin  

msd  >>  mad
pagalni  >>  Paganini
rit  >>  right
knw  >>  kn
brokeup  >>  broke up
coz  >>  cox
dont  >>  font
tatoos  >>  tattoos
siri  >>  sir
ned  >>  end
yess  >>  yetis
erts  >>  erst
nup  >>  pun
taylor  >>  Taylor
api  >>  API
api  >>  API
kannada  >>  Kannada
dont  >>  font
yrr  >>  yer
dont  >>  font
asian  >>  Asian
qstn  >>  Stan
abt  >>  bat
knd  >>  kn
actrss  >>  actress
pls  >>  pl
marru  >>  marry
favourite  >>  favorite
rit  >>  right
wat  >>  eat
raa  >>  raga
areu  >>  urea
andy  >>  Andy
andy  >>  Andy
rubin  >>  Rubin
yar  >>  ray
aapn  >>  nap
dont  >>  font
smlie  >>  smile
fuckin  >>  buckskin
yayy  >>  Maya
babs  >>  nabs
eng  >>  neg
pls  >>  pl
bandra  >>  Sandra
mumbai  >>  Mumbai
bandra  >>  Sandra
ert  >>  err
dont  >>  font
febret  >>  ferret
bestfriend  >>  best friend
praveen  >>  raven
sse  >>  SSE
luv  >>  lav
monday  >>  Monday
lyk  >>  lye
onwards  >>  inwards
luv  >>  lav
iam  >>  aim
dont  >>  font
frt  >>  fry
der  >>  red
abt  >>  bat
lols  >> 

didnt  >>  dint
urs  >>  yrs
soo  >>  sew
telematic  >>  emblematic
urs  >>  yrs
yas  >>  say
aer  >>  ear
aer  >>  ear
lol  >>  lo
doin  >>  don
erience  >>  experience
sry  >>  yrs
dont  >>  font
naa  >>  baa
knoww  >>  know
bdw  >>  bow
sngry  >>  angry
yweet  >>  tweet
india  >>  India
0pm  >>  pm
plz  >>  pl
oky  >>  okay
iam  >>  aim
abt  >>  bat
dont  >>  font
conte  >>  conch
llb  >>  LLB
wbu  >>  bu
dumbass  >>  dumb ass
callin  >>  calling
wat  >>  eat
ressing  >>  dressing
twelebrity  >>  celebrity
dont  >>  font
plz  >>  pl
luv  >>  lav
gottisim  >>  glottis
dont  >>  font
nicky  >>  nick
minage  >>  mintage
bff  >>  BFF
soo  >>  sew
bff  >>  BFF
ranbir  >>  Iranian
kapoor  >>  Kickapoo
eriences  >>  experiences
cuz  >>  cuss
sanke  >>  snake
okk  >>  oik
india  >>  India
india  >>  India
soo  >>  sew
dont  >>  font
knw  >>  kn
believeable  >>  believable
madkolr  >>  matador
ela  >>  lea
jabardast  >>  bastard
dumbass  >>  dumb ass
wbu  >>  bu
okey  >>  key
pls  >>  pl
don

dont  >>  font
favourite  >>  favorite
evry  >>  very
yay  >>  ya
sem  >>  chem
sirius  >>  Sirius
dont  >>  font
aww  >>  awe
youu  >>  you
coz  >>  cox
alvida  >>  Vidal
reall  >>  real
pre  >>  per
witg  >>  with
wifi  >>  wife
knw  >>  kn
dng  >>  ding
knw  >>  kn
knw  >>  kn
knw  >>  kn
yaar  >>  year
acknowledgement  >>  acknowledgment
pls  >>  pl
padmavat  >>  palmate
noo  >>  new
dua  >>  dual
lipa  >>  lips
dont  >>  font
mxm  >>  mm
imittate  >>  imitate
fema  >>  fame
desaid  >>  said
pune  >>  pine
viju  >>  Vitus
oow  >>  woo
didnt  >>  dint
frnd  >>  fend
becaz  >>  became
bangalore  >>  Bangalore
monday  >>  Monday
tuesday  >>  Tuesday
saturday  >>  Saturday
monday  >>  Monday
microsoft  >>  Microsoft
peoblem  >>  problem
pagal  >>  pagan
jus  >>  us
noo  >>  new
tis  >>  tees
seriou  >>  serious
img  >>  mg
dnt  >>  dent
reall  >>  real
macys  >>  macs
knw  >>  kn
siri  >>  sir
rena  >>  Rena
wasnt  >>  wast
yayy  >>  Maya
imma  >>  imam
sry  >>  yrs
iam  >>  aim
wat  >

yas  >>  say
sittin  >>  sitting
aww  >>  awe
friday  >>  Friday
realease  >>  release
lols  >>  lolls
abt  >>  bat
woww  >>  wow
aww  >>  awe
dont  >>  font
favourite  >>  favorite
bollywood  >>  Bollywood
mondays  >>  Mondays
alrite  >>  literal
indian  >>  Indian
lanae  >>  lane
yall  >>  tall
fav  >>  fab
msg  >>  MSG
msg  >>  MSG
sooper  >>  super
foody  >>  goody
india  >>  India
pawala  >>  paywall
punjabi  >>  Punjabi
abt  >>  bat
woow  >>  woo
floyd  >>  Floyd
floyd  >>  Floyd
favourite  >>  favorite
btwn  >>  BTW
wer  >>  wee
konsa  >>  koans
batao  >>  Bataan
dont  >>  font
umm  >>  mum
bollywood  >>  Bollywood
forgt  >>  fort
linux  >>  Linux
microsoft  >>  Microsoft
itz  >>  it
comsidered  >>  considered
isn  >>  sin
bestfriend  >>  best friend
nop  >>  bop
didnt  >>  dint
adenoma  >>  nomad
dont  >>  font
lil  >>  ill
autocorrected  >>  auto corrected
fosure  >>  foreclosure
tommorow  >>  tomorrow
yor  >>  tor
programme  >>  programmer
feeded  >>  deeded
informations  >> 

ntg  >>  mtg
plzz  >>  plaza
meesengr  >>  messenger
plzz  >>  plaza
dnt  >>  dent
soo  >>  sew
msg  >>  MSG
dont  >>  font
dont  >>  font
yaar  >>  year
jim  >>  Jim
carey  >>  Carey
sry  >>  yrs
coz  >>  cox
warframe  >>  war frame
ofit  >>  oft
alein  >>  alien
werid  >>  weird
ikr  >>  irk
soorryy  >>  sorry
plese  >>  please
plz  >>  pl
bollywood  >>  Bollywood
ress  >>  reds
dnt  >>  dent
coz  >>  cox
sende  >>  dense
awsm  >>  swam
iam  >>  aim
friday  >>  Friday
kfc  >>  KFC
aww  >>  awe
yong  >>  Yong
selfiee  >>  selfie
imagemagick  >>  magicking
imagemagick  >>  magicking
sierri  >>  sierra
distrubed  >>  disturbed
fullfill  >>  fulfill
goin  >>  goon
bff  >>  BFF
bff  >>  BFF
todays  >>  today
tiddi  >>  tiddly
wat  >>  eat
dele  >>  delve
alli  >>  all
september  >>  September
wat  >>  eat
iam  >>  aim
dont  >>  font
noo  >>  new
yaar  >>  year
ect  >>  etc
abt  >>  bat
dnt  >>  dent
freind  >>  friend
nokia  >>  Nokia
lool  >>  look
pls  >>  pl
frm  >>  fem
utell  >>  tel

robo  >>  rob
skys  >>  sky
dont  >>  font
lawl  >>  wall
gannam  >>  Annam
bollywood  >>  Bollywood
bollywood  >>  Bollywood
musi  >>  mus
cuz  >>  cuss
alls  >>  all
tellb  >>  tell
plz  >>  pl
bai  >>  bi
ganpati  >>  Galatians
enquiry  >>  inquiry
nollywood  >>  Hollywood
skil  >>  ski
dont  >>  font
yoi  >>  you
mxm  >>  mm
realising  >>  realizing
wbu  >>  bu
dont  >>  font
dont  >>  font
abt  >>  bat
yess  >>  yetis
aww  >>  awe
ensive  >>  endive
mercedes  >>  Mercedes
okk  >>  oik
microsoft  >>  Microsoft
microsoft  >>  Microsoft
agr  >>  gar
microsoft  >>  Microsoft
oxymorons  >>  oxymoron
microsoft  >>  Microsoft
elses  >>  else
wat  >>  eat
soo  >>  sew
wat  >>  eat
umm  >>  mum
plz  >>  pl
okk  >>  oik
dont  >>  font
meanin  >>  melanin
wat  >>  eat
birday  >>  birthday
frind  >>  grind
interestingi  >>  interesting
abt  >>  bat
programing  >>  programming
dont  >>  font
fellings  >>  felling
sence  >>  seance
coz  >>  cox
dats  >>  tads
dats  >>  tads
sravani  >>  caravan

grt  >>  get
dont  >>  font
gifs  >>  figs
creat  >>  crest
iam  >>  aim
wud  >>  wood
bollywood  >>  Bollywood
wwe  >>  ewe
aww  >>  awe
pjs  >>  pj's
bcoz  >>  cozy
sweety  >>  sweet
tryin  >>  trying
america  >>  America
erience  >>  experience
benaz  >>  benzene
rommate  >>  roommate
rommate  >>  roommate
bollywood  >>  Bollywood
cmng  >>  coming
emma  >>  Emma
sam  >>  SAM
numbar  >>  number
msg  >>  MSG
bombay  >>  Bombay
ranbir  >>  Iranian
jann  >>  jinn
decreats  >>  decrease
yaa  >>  ya
goa  >>  Goa
pokémons  >>  Pokemon
december  >>  December
dont  >>  font
okayy  >>  okay
fuckin  >>  buckskin
soni  >>  ions
wts  >>  qts
taI  >>  ta
microsoft  >>  Microsoft
favourite  >>  favorite
bffs  >>  biffs
microsoft  >>  Microsoft
sydney  >>  Sydney
lulz  >>  lull
courtney  >>  Courtney
dont  >>  font
soo  >>  sew
lololol  >>  lollop
iima  >>  Lima
dont  >>  font
dont  >>  font
dont  >>  font
abilitity  >>  ability
bcz  >>  biz
noo  >>  new
dexter  >>  Dexter
dnt  >>  dent
knw  >>  kn

KeyboardInterrupt: 

In [387]:
from tokenizer import tokenizer

T = tokenizer.TweetTokenizer(regularize=True)

[T.tokenize(sentence) for sentence in sentences_without_elongations]

[['do',
  'not',
  'worry',
  'I',
  'am',
  'girl',
  'hmm',
  'how',
  'do',
  'i',
  'know',
  'if',
  'you',
  'are',
  "what's",
  'ur',
  'name',
  '?'],
 ['when',
  'did',
  'i',
  '?',
  'saw',
  'many',
  'times',
  'i',
  'think',
  '-',
  '_',
  '-',
  'no',
  '.',
  'i',
  'never',
  'saw',
  'you'],
 ['by', 'by', 'google', 'chrome', 'where', 'you', 'live'],
 ['you',
  'r',
  'ridiculous',
  'i',
  'might',
  'be',
  'ridiculous',
  'but',
  'i',
  'am',
  'telling',
  'the',
  'truth',
  '.',
  'you',
  'little',
  'disgusting',
  'whore'],
 ['just',
  'for',
  'time',
  'pass',
  'wt',
  'do',
  'you',
  'do',
  '4',
  'a',
  'living',
  'then',
  'maybe'],
 ['I', 'am', 'a', 'dog', 'person', 'you', 'are', 'so', 'rude', 'whaat', 'why'],
 ['so',
  'whatsup',
  'nothing',
  'much',
  '.',
  'sitting',
  'sipping',
  'and',
  'watching',
  'tv',
  '.',
  'how',
  'abt',
  'you',
  'what',
  'are',
  'you',
  'watching',
  'on',
  'tv',
  '?'],
 ['ok', 'ok', 'I', 'am', 'back',